# 📈 Baseline Models
- Persistence baseline (next week = last week)
- XGBoost with engineered features
- Evaluate MAE / RMSE / MAPE / Peak MAE
- Save models, metrics, and training loss


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, json, time, numpy as np, pandas as pd, joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import warnings; warnings.filterwarnings('ignore')

BASE_DIR  = '/content/drive/MyDrive/Electricity_Load_Forecast'
MODEL_DIR = os.path.join(BASE_DIR, 'models')

In [ ]:
config         = joblib.load(os.path.join(MODEL_DIR, 'config.pkl'))
target_scaler  = joblib.load(os.path.join(MODEL_DIR, 'target_scaler.pkl'))

X_train_w = np.load(os.path.join(MODEL_DIR, 'X_train_w.npy'))
y_train_w = np.load(os.path.join(MODEL_DIR, 'y_train_w.npy'))
X_val_w   = np.load(os.path.join(MODEL_DIR, 'X_val_w.npy'))
y_val_w   = np.load(os.path.join(MODEL_DIR, 'y_val_w.npy'))
X_test_w  = np.load(os.path.join(MODEL_DIR, 'X_test_w.npy'))
y_test_w  = np.load(os.path.join(MODEL_DIR, 'y_test_w.npy'))

LOAD_IDX   = config['load_col_idx']
INPUT_LEN  = config['INPUT_LEN']
OUTPUT_LEN = config['OUTPUT_LEN']

print(f"Train: {X_train_w.shape}, Val: {X_val_w.shape}, Test: {X_test_w.shape}")
print(f"Load column index: {LOAD_IDX}")

In [ ]:
def inverse_scale(y_scaled):
    """Inverse-transform scaled load values back to MW."""
    return target_scaler.inverse_transform(y_scaled.reshape(-1, 1)).reshape(y_scaled.shape)

def compute_metrics(y_true, y_pred, label=""):
    """Compute MAE, RMSE, MAPE, Peak MAE on original-scale values."""
    y_t = inverse_scale(y_true)
    y_p = inverse_scale(y_pred)
    mae  = mean_absolute_error(y_t.flatten(), y_p.flatten())
    rmse = np.sqrt(mean_squared_error(y_t.flatten(), y_p.flatten()))
    mask = y_t.flatten() != 0
    mape = np.mean(np.abs((y_t.flatten()[mask] - y_p.flatten()[mask]) / y_t.flatten()[mask])) * 100
    # Peak MAE: error at peak load hour in each window
    peak_errors = [np.abs(y_t[i, np.argmax(y_t[i])] - y_p[i, np.argmax(y_t[i])]) for i in range(len(y_t))]
    peak_mae = np.mean(peak_errors)
    metrics = {'MAE': round(mae, 2), 'RMSE': round(rmse, 2),
               'MAPE': round(mape, 2), 'Peak_MAE': round(peak_mae, 2)}
    if label:
        print(f"\n{'='*40}\n{label}\n{'='*40}")
        for k, v in metrics.items():
            print(f"  {k:10s}: {v}")
    return metrics

In [ ]:
#  1) PERSISTENCE BASELINE — next week = last week
# ═══════════════════════════════════════════════
def persistence_predict(X_windows, load_idx):
    """Predict next 168h = last 168h of observed load in the input window."""
    return X_windows[:, :, load_idx]  # (N, 168) — the load from input window

y_pred_persist_val  = persistence_predict(X_val_w, LOAD_IDX)
y_pred_persist_test = persistence_predict(X_test_w, LOAD_IDX)

persist_val  = compute_metrics(y_val_w,  y_pred_persist_val,  "Persistence — Validation")
persist_test = compute_metrics(y_test_w, y_pred_persist_test, "Persistence — Test")

In [ ]:
#  2) XGBOOST WITH ENGINEERED FEATURES
# ═══════════════════════════════════════════════
def engineer_features(X_windows, load_idx):
    """Extract summary statistics from each input window for XGBoost."""
    feat_list = []
    for w in X_windows:
        f = []
        load = w[:, load_idx]
        # Load statistics (9 features)
        f.extend([load.mean(), load.std(), load.min(), load.max(),
                  load[-1], load[0], load[-1] - load[0],
                  load[-24:].mean(), load[-48:].mean()])
        # Other features: mean + last value (2 per column)
        for j in range(w.shape[1]):
            if j != load_idx:
                f.extend([w[:, j].mean(), w[-1, j]])
        feat_list.append(f)
    return np.array(feat_list, dtype=np.float32)

print("\nEngineering XGBoost features...")
t0 = time.time()
Xf_train = engineer_features(X_train_w, LOAD_IDX)
Xf_val   = engineer_features(X_val_w,   LOAD_IDX)
Xf_test  = engineer_features(X_test_w,  LOAD_IDX)
print(f"  Done in {time.time()-t0:.1f}s — shape: {Xf_train.shape}")

In [ ]:
print("\nTraining XGBoost (multi-output, this may take a few minutes)...")
t0 = time.time()

xgb_model = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist',       # Use 'gpu_hist' if GPU available
    random_state=42,
    verbosity=1
)

# XGBoost supports multi-output when y has shape (n_samples, n_outputs)
xgb_model.fit(
    Xf_train, y_train_w,
    eval_set=[(Xf_val, y_val_w)],
    verbose=50
)
print(f"XGBoost trained in {time.time()-t0:.1f}s")

In [ ]:
y_pred_xgb_val  = xgb_model.predict(Xf_val)
y_pred_xgb_test = xgb_model.predict(Xf_test)

xgb_val  = compute_metrics(y_val_w,  y_pred_xgb_val,  "XGBoost — Validation")
xgb_test = compute_metrics(y_test_w, y_pred_xgb_test, "XGBoost — Test")

In [ ]:
evals = xgb_model.evals_result()
if evals:
    val_key = list(evals.keys())[0]
    loss_key = list(evals[val_key].keys())[0]
    xgb_loss = evals[val_key][loss_key]
    plt.figure(figsize=(10, 4))
    plt.plot(xgb_loss, label='XGBoost Val Loss')
    plt.xlabel('Boosting Round'); plt.ylabel('Loss'); plt.title('XGBoost Training Curve')
    plt.legend(); plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(MODEL_DIR, 'xgb_training_loss.png'), dpi=150)
    plt.show()

In [ ]:
joblib.dump(xgb_model, os.path.join(MODEL_DIR, 'xgb_model.pkl'))

baseline_metrics = {
    'Persistence': {'val': persist_val, 'test': persist_test},
    'XGBoost':     {'val': xgb_val,     'test': xgb_test}
}
with open(os.path.join(MODEL_DIR, 'baseline_metrics.json'), 'w') as f:
    json.dump(baseline_metrics, f, indent=2)

print("\n✅ Saved: xgb_model.pkl, baseline_metrics.json, xgb_training_loss.png")

In [ ]:
print("\n" + "="*65)
print("BASELINE COMPARISON (Test Set)")
print("="*65)
print(f"{'Model':<15} {'MAE':>8} {'RMSE':>8} {'MAPE%':>8} {'PeakMAE':>10}")
print("-"*65)
for model_name, m in baseline_metrics.items():
    t = m['test']
    print(f"{model_name:<15} {t['MAE']:>8} {t['RMSE']:>8} {t['MAPE']:>8} {t['Peak_MAE']:>10}")
print("="*65)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
for idx, ax in enumerate(axes.flat):
    sample_idx = idx * (len(y_test_w) // 4)
    y_true_inv = inverse_scale(y_test_w[sample_idx:sample_idx+1])[0]
    y_pers_inv = inverse_scale(y_pred_persist_test[sample_idx:sample_idx+1])[0]
    y_xgb_inv  = inverse_scale(y_pred_xgb_test[sample_idx:sample_idx+1])[0]
    ax.plot(y_true_inv, 'k-', lw=2, label='Actual')
    ax.plot(y_pers_inv, 'r--', alpha=0.7, label='Persistence')
    ax.plot(y_xgb_inv,  'b--', alpha=0.7, label='XGBoost')
    ax.set_title(f'Test Sample #{sample_idx}')
    ax.set_xlabel('Hour'); ax.set_ylabel('Load (MW)')
    ax.legend(); ax.grid(True, alpha=0.3)
plt.suptitle('Baseline Predictions vs Actual (Test Set)', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'baseline_predictions.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✅ Done! Proceed to deep_learning_model notebook.")